# Electroprophet⚡️

In this notebook, I will do the first steps on our final project for Le Wagon's Data Science bootcamp. Here, I'll try to model the carbon emission in France using weather data, in order to provide recomendations for users to reduce their carbon footprint 👣.

## Getting the data (outdated, don't use it)

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Wheater API

Here we'll be able to get weather data of a chosen city. 

As the energy data is given by region, later we'll have to check how many cities we have to take in consideration in order to get the weather in a given region.

In [ ]:
import requests
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import os.path

# Get the absolute path of the directory where the script is located
script_dir = os.getcwd()

# Construct the path to the "my_directory" directory relative to the script directory
raw_data_path = os.path.join(script_dir, "raw_data")

In [ ]:
def get_weather(city, years=10, overwrite=False):
    
    '''
    This function receives the name of a city and a number of years, and returns a dataframe 
    with weather data from this city during those past years
    '''
    
    path = raw_data_path + '/df_' + city.lower() + '_weather.csv'
    file_exists = os.path.isfile(path) 
    
    if file_exists and not overwrite:
        
        print('Found a file for this city. Importing...')
        
        weather_df = pd.read_csv(path, index_col=0)
        
    else:
        
        print('Creating a new .csv file for this city')
        
        # First we declare the weather parameters. Here we'll be taking all params supported by the API
        weather_params = ['temperature_2m','relativehumidity_2m','dewpoint_2m',
                      'apparent_temperature','pressure_msl','surface_pressure',
                      'precipitation','rain','snowfall','cloudcover',
                      'cloudcover_low','cloudcover_mid','cloudcover_high',
                      'shortwave_radiation','direct_radiation','direct_normal_irradiance',
                      'diffuse_radiation','windspeed_10m','windspeed_100m',
                      'winddirection_10m','winddirection_100m','windgusts_10m',
                      'et0_fao_evapotranspiration','weathercode','vapor_pressure_deficit',
                      'soil_temperature_0_to_7cm','soil_temperature_7_to_28cm',
                      'soil_temperature_28_to_100cm','soil_temperature_100_to_255cm',
                      'soil_moisture_0_to_7cm','soil_moisture_7_to_28cm',
                      'soil_moisture_28_to_100cm','soil_moisture_100_to_255cm']

        # This request is done in order to get the latitude and longitude of the desired city
        city_response = requests.get('https://geocoding-api.open-meteo.com/v1/search',
                           params = {'name': city}).json()

        lat = city_response['results'][0]['latitude']
        lon = city_response['results'][0]['longitude']

        # Then we compute the dates used to get the weather data
        ## The API only has data until 9 days ago
        end_date = (datetime.date.today() - relativedelta(days=8)).strftime('%Y-%m-%d') 
        #start_date = (datetime.date.today() - relativedelta(years=years)).strftime('%Y-%m-%d')
        start_date = '2013-01-01 13:00:00'

        # So we make the request to the weather API archive
        weather_response = requests.get('https://archive-api.open-meteo.com/v1/archive',
                           params = {'latitude': lat,
                                    'longitude': lon,
                                    'start_date': start_date,
                                    'end_date': end_date,
                                    'hourly': weather_params,
                                    'timezone': 'auto'}).json()
        
        weather_df = pd.DataFrame(weather_response['hourly'], columns = ['time'] + weather_params)
        weather_df['time'] = pd.to_datetime(weather_df['time'], format='%Y-%m-%d')
        weather_df = weather_df.set_index('time')
        
        weather_df.to_csv(path)
    
    print('Done ✅')
    return weather_df

In [ ]:
from prophecy import data

data.get_weather('Amiens')
#df = data.get_weather('Paris',overwrite=True)
#df = data.get_weather('Marseille',overwrite=True)

ImportError: cannot import name 'data' from 'prophecy' (/home/jupyter/code/caiovnd/electroprophet/prophecy/__init__.py)

In [ ]:
import pandas as pd
pd.to_datetime(df.index, format='%Y-%m-%d')

In [ ]:
import datetime

print(datetime.datetime.strptime('2013-01-01','%Y-%m-%d'))

First let's explore and see if we find any problems

In [ ]:
df.duplicated().sum() # Checking for duplicates

In [ ]:
df.isnull().sum().sort_values(ascending=False)/len(df) # Checking for null values

In [ ]:
weather_params = ['temperature_2m','relativehumidity_2m','dewpoint_2m',
                      'apparent_temperature','pressure_msl','surface_pressure',
                      'precipitation','rain','snowfall','cloudcover',
                      'cloudcover_low','cloudcover_mid','cloudcover_high',
                      'shortwave_radiation','direct_radiation','direct_normal_irradiance',
                      'diffuse_radiation','windspeed_10m','windspeed_100m',
                      'winddirection_10m','winddirection_100m','windgusts_10m',
                      'et0_fao_evapotranspiration','weathercode','vapor_pressure_deficit',
                      'soil_temperature_0_to_7cm','soil_temperature_7_to_28cm',
                      'soil_temperature_28_to_100cm','soil_temperature_100_to_255cm',
                      'soil_moisture_0_to_7cm','soil_moisture_7_to_28cm',
                      'soil_moisture_28_to_100cm','soil_moisture_100_to_255cm']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

Then we can preprocess our data

### 1. MinMax Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
df_preproc = df.copy()
for feature in weather_params:
    scaler.fit(df[[feature]])
    df_preproc[feature] = scaler.transform(df[[feature]])

In [ ]:
df_preproc.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

index = 0

for index in enumerate(weather_params):
    index=index[0]
    fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(12,3))

    sns.histplot(df[weather_params[index]], bins=200,kde = True, ax=axes[0]);
    sns.boxplot(data=df, x=weather_params[index], ax=axes[1]);
    sns.histplot(df_preproc[weather_params[index]], bins=200,kde = True, ax=axes[2], color='orange');
    sns.boxplot(data=df_preproc, x=weather_params[index], ax=axes[3], color='orange');

### 2. Robust Scaler

In [ ]:
from sklearn.preprocessing import RobustScaler


rb_scaler = RobustScaler() 
df_preproc2 = df.copy()
for feature in weather_params:
    rb_scaler.fit(df[[feature]])
    df_preproc2[feature] = rb_scaler.transform(df[[feature]])

rb_scaler.fit(data[['GrLivArea']]) 

data['GrLivArea'] = rb_scaler.transform(data[['GrLivArea']]) 


In [ ]:
df_weather = data.get_weather('Amiens')
df_weather

In [ ]:
df_merge = data.merge_weather_energy_df('Amiens', 'Hauts-de-France', 'eolien')
df_merge

In [ ]:
prod_df = data.get_energy_production(-1, 0, 'Hauts-de-France')

In [ ]:
prod_df.columns

## Baseline model - Random Forest (not the best parameters)

First we import our class:

In [ ]:
from prophecy.data_3_cities import WeatherEnergy

In [ ]:
data = WeatherEnergy(limit=-1, offset=0, refine='Hauts-de-France', city1='Heudicourt', city2='Bucy-les-Pierrepont', city3='Riencourt', target='eolien')

Then we get our dataframe

In [ ]:
df = data.merged()
df.head()

And we drop all `NaN`'s from our target

In [ ]:
df_nona=df.dropna(subset=['eolien'])

Now we define our features and target

In [ ]:
X = df_nona.drop(columns=['eolien','weathercode'])
y = df_nona['eolien']

Then we run and evaluate our baseline model with RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
forest = RandomForestRegressor(max_depth=10, n_estimators=100)
score_baseline = cross_val_score(forest, X, y, cv=5,scoring='r2',n_jobs=-1)
score_baseline.mean()

Let's try also for XGBoost

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb_reg = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)
score_baseline = cross_val_score(xgb_reg, X, y, cv=5,scoring='r2',n_jobs=-1)
score_baseline.mean()

## Model Tuning - Random Forest (here I run GridSearch and use the best parameters)

In [ ]:
import numpy as np
import pandas as pd
from random import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import multiprocessing

In [ ]:
df_scaled = pd.read_csv('raw_data/wind_energy_scaled.csv',index_col=0)
df_scaled.head()

In [ ]:
df_scaled.columns

In [ ]:
X_scaled = df_scaled.drop(columns=['eolien'])
y = df_scaled['eolien']

In [ ]:
X_red = X_scaled.tail(8760)
y_red = y.tail(8760)

In [ ]:
forest = RandomForestRegressor(max_depth=10, n_estimators=100)
score_scaled = cross_val_score(forest, X_red, y_red, cv=5,scoring='r2',n_jobs=-1)
score_scaled.mean()

In [ ]:
# Define the parameter grid to search over
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the RandomForestRegressor model
rf_model = RandomForestRegressor(random_state=42)

In [ ]:
# Use RandomizedSearchCV to search for the best hyperparameters
random_search = RandomizedSearchCV(rf_model, param_distributions=param_grid, n_iter=50, cv=5, n_jobs=multiprocessing.cpu_count(), scoring='r2')

In [ ]:
# Fit the model on the Boston Housing dataset
random_search.fit(X_red, y_red)

In [ ]:
# Print the best hyperparameters found
print('Best hyperparameters:', random_search.best_params_)

Results from last run: (n_estimators: 150, min_samples_split: 2, min_samples_leaf: 2, max_depth: 10)

In [ ]:
# Print the best score found
print('Best score:', random_search.best_score_)

## Feature importance

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.inspection import permutation_importance

Here I use the .csv without the vector features

In [ ]:
df_scaled = pd.read_csv('raw_data/wind_energy_scaled.csv',index_col=0)

Here I am dropping the worst features

In [ ]:
X_scaled = df_scaled.drop(columns=['eolien','cloudcover','cloudcover_low','cloudcover_mid',
                       'cloudcover_high','shortwave_radiation','direct_radiation',
                      'direct_normal_irradiance','diffuse_radiation','et0_fao_evapotranspiration',
                       'soil_temperature_0_to_7cm','soil_temperature_7_to_28cm','soil_temperature_28_to_100cm',
                       'soil_temperature_100_to_255cm','soil_moisture_0_to_7cm','soil_moisture_7_to_28cm',
                       'soil_moisture_28_to_100cm','soil_moisture_100_to_255cm','dewpoint_2m','precipitation',
                        'rain','snowfall'])
y = df_scaled['eolien']

In [ ]:
X_red_train = X_scaled.iloc[:8760]
y_red_train = y.iloc[:8760]
X_red_test = X_scaled.iloc[8760:8784]
y_red_test = y.iloc[8760:8784]

In [ ]:
forest = RandomForestRegressor(n_estimators=150, min_samples_split=2, min_samples_leaf=2, max_depth=10)
result = forest.fit(X_red_train,y_red_train)

In [ ]:
result = permutation_importance(forest, X_red_test, y_red_test.values, n_repeats=10, random_state=42, scoring='r2')

In [ ]:
df = pd.DataFrame({
    'feature': X_red_test.columns,
    'importance_mean': abs(result['importances_mean']),
    'importance_std': result['importances_std']
}).sort_values('importance_mean', ascending=False)

print(df)

## Best RandomForest

In [ ]:
df_scaled = pd.read_csv('raw_data/wind_energy_scaled.csv',index_col=0)

In [ ]:
X_scaled = df_scaled.drop(columns=['eolien','cloudcover','cloudcover_low','cloudcover_mid',
                       'cloudcover_high','shortwave_radiation','direct_radiation',
                      'direct_normal_irradiance','diffuse_radiation','et0_fao_evapotranspiration',
                       'soil_temperature_0_to_7cm','soil_temperature_7_to_28cm','soil_temperature_28_to_100cm',
                       'soil_temperature_100_to_255cm','soil_moisture_0_to_7cm','soil_moisture_7_to_28cm',
                       'soil_moisture_28_to_100cm','soil_moisture_100_to_255cm','dewpoint_2m','precipitation',
                        'rain','snowfall'])
y = df_scaled['eolien']

Training one year to predict the following day

In [ ]:
X_red_train = X_scaled.iloc[:8760]
y_red_train = y.iloc[:8760]
X_red_test = X_scaled.iloc[8760:8784]
y_red_test = y.iloc[8760:8784]

Using the best params from GridSearch

In [ ]:
forest = RandomForestRegressor(n_estimators=150, min_samples_split=2, min_samples_leaf=2, max_depth=10)
score_scaled = cross_val_score(forest, X_red_train, y_red_train, cv=5,scoring='r2',n_jobs=-1)
score_scaled.mean()

## SARIMAX (gave up on it)

In [ ]:
from pmdarima.arima import auto_arima
import statsmodels.api as sm
import numpy as np
import pandas as pd
from random import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import multiprocessing
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
df_scaled = pd.read_csv('raw_data/wind_energy_scaled.csv',index_col=0)

In [ ]:
X_scaled = df_scaled.drop(columns=['eolien','cloudcover','cloudcover_low','cloudcover_mid',
                       'cloudcover_high','shortwave_radiation','direct_radiation',
                      'direct_normal_irradiance','diffuse_radiation','et0_fao_evapotranspiration',
                       'soil_temperature_0_to_7cm','soil_temperature_7_to_28cm','soil_temperature_28_to_100cm',
                       'soil_temperature_100_to_255cm','soil_moisture_0_to_7cm','soil_moisture_7_to_28cm',
                       'soil_moisture_28_to_100cm','soil_moisture_100_to_255cm','dewpoint_2m','precipitation',
                        'rain','snowfall'])
y = df_scaled['eolien']

In [ ]:
X_red_train = X_scaled.iloc[:8760]
y_red_train = y.iloc[:8760]
X_red_test = X_scaled.iloc[8760:8784]
y_red_test = y.iloc[8760:8784]

In [ ]:
# Define the SARIMAX model with exogenous variables
model = sm.tsa.statespace.SARIMAX(endog=y_red_train, exog=X_red_train, order=(2,1,1), seasonal_order=(1,0,0,12))

# Use auto_arima to find the best hyperparameters for the SARIMAX model
arima_model = auto_arima(y_red_train, exogenous=X_red_train, seasonal=True, m=24, 
                         max_p=2, max_d=2, max_q=2, max_P=2, max_D=1, max_Q=2, 
                         stepwise=False, trace=True, information_criterion='bic',
                         parallel=True,njobs=-1)

# Print the best hyperparameters found by auto_arima
print(arima_model.summary())

In [ ]:
# Fit the SARIMAX model using the best hyperparameters found by auto_arima
sarima = SARIMAX(endog=y_red_train, exog=X_red_train, order=(0, 1, 2), seasonal_order=(0, 0, 0, 24))
results = sarima.fit()

In [ ]:
y_red_test

In [ ]:
# Make predictions on the test data using the trained model
predictions = results.predict(start='2014-03-13 00:00:00', end='2014-03-13 23:00:00', exog=X_red_test)
predictions

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
# Calculate the score of the forecast
score = r2_score(y_red_test, predictions)
print('Forecast score:', score)

Don't forget to move this notebook back to the notebooks folder using import sys

## RNN

In [1]:
%load_ext autoreload
%autoreload 2

### Importations

In [11]:
# Data manipulation
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os

# Deep Learning
import tensorflow as tf

from typing import Dict, List, Tuple, Sequence

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.layers.experimental.preprocessing import Normalization

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

import sys
sys.path.append('../')
from prophecy.get_data import WeatherEnergy
from prophecy.feature_processing import FeaturePreprocessing
import numpy as np

import math

### Functions

In [12]:
def get_folds(
    df: pd.DataFrame,
    fold_length: int,
    fold_stride: int) -> List[pd.DataFrame]:
    """    
    This function slides through the Time Series dataframe of shape (n_timesteps, n_features) to create folds
    - of equal `fold_length`
    - using `fold_stride` between each fold

    Args:
        df (pd.DataFrame): Overall dataframe
        fold_length (int): How long each fold should be in rows
        fold_stride (int): How many timesteps to move forward between taking each fold

    Returns:
        List[pd.DataFrame]: A list where each fold is a dataframe within
    """
    # $CHALLENGIFY_BEGIN
    
    folds = []
    for idx in range(0, len(df), fold_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (idx + fold_length) > len(df):
            break
        fold = df.iloc[idx:idx + fold_length, :]
        folds.append(fold)
    return folds

In [13]:
def train_test_split(fold:pd.DataFrame,
                     train_test_ratio: float,
                     input_length: int) -> Tuple[pd.DataFrame]:
    """From a fold dataframe, take a train dataframe and test dataframe based on 
    the split ratio.
    - df_train should contain all the timesteps until round(train_test_ratio * len(fold))
    - df_test should contain all the timesteps needed to create all (X_test, y_test) tuples

    Args:
        fold (pd.DataFrame): A fold of timesteps
        train_test_ratio (float): The ratio between train and test 0-1
        input_length (int): How long each X_i will be

    Returns:
        Tuple[pd.DataFrame]: A tuple of two dataframes (fold_train, fold_test)
    """
    # $CHALLENGIFY_BEGIN
    
    # TRAIN SET
    # ======================
    last_train_idx = round(train_test_ratio * len(fold))
    fold_train = fold.iloc[0:last_train_idx, :]

    # TEST SET
    # ======================    
    first_test_idx = last_train_idx - input_length
    fold_test = fold.iloc[first_test_idx:, :]

    return (fold_train, fold_test)

In [14]:
def get_Xi_yi(
    fold:pd.DataFrame, 
    input_length:int, 
    output_length:int) -> Tuple[pd.DataFrame]:
    """given a fold, it returns one sequence (X_i, y_i) as based on the desired 
    input_length and output_length with the starting point of the sequence being chosen at random based

    Args:
        fold (pd.DataFrame): A single fold
        input_length (int): How long each X_i should be 
        output_length (int): How long each y_i should be

    Returns:
        Tuple[pd.DataFrame]: A tuple of two dataframes (X_i, y_i)
    """
    # $CHALLENGIFY_BEGIN
    first_possible_start = 0
    last_possible_start = len(fold) - (input_length + output_length) + 1
    random_start = np.random.randint(first_possible_start, last_possible_start)
    X_i = fold.iloc[random_start:random_start+input_length].drop(columns=TARGET)    
    y_i = fold.iloc[random_start+input_length:
                  random_start+input_length+output_length][[TARGET]]
    
    return (X_i, y_i)

In [15]:
def get_X_y(
    fold:pd.DataFrame,
    number_of_sequences:int,
    input_length:int,
    output_length:int) -> Tuple[np.array]:
    """Given a fold generate X and y based on the number of desired sequences 
    of the given input_length and output_length

    Args:
        fold (pd.DataFrame): Fold dataframe
        number_of_sequences (int): The number of X_i and y_i pairs to include
        input_length (int): Length of each X_i
        output_length (int): Length of each y_i

    Returns:
        Tuple[np.array]: A tuple of numpy arrays (X, y)
    """
    # $CHALLENGIFY_BEGIN    
    X, y = [], []

    for i in range(number_of_sequences):
        (Xi, yi) = get_Xi_yi(fold, input_length, output_length)
        X.append(Xi)
        y.append(yi)
        
    return np.array(X), np.array(y)

In [16]:
def init_model(X_train, y_train):
    
    # $CHALLENGIFY_BEGIN    
    
    # 0 - Normalization
    # ======================    
    normalizer = Normalization()
    normalizer.adapt(X_train)
    
    # 1 - RNN architecture
    # ======================    
    model = models.Sequential()
    ## 1.0 - All the rows will be standardized through the already adapted normalization layer
    model.add(normalizer)
    ## 1.1 - Recurrent Layer
    model.add(layers.LSTM(64, 
                          activation='tanh', 
                          return_sequences = False,
                          kernel_regularizer=L1L2(l1=0.05, l2=0.05),
                          ))
    ## 1.2 - Predictive Dense Layers
    output_length = y_train.shape[1]
    model.add(layers.Dense(output_length, activation='linear'))

    # 2 - Compiler
    # ======================    
    adam = optimizers.Adam(learning_rate=0.02)    
    model.compile(loss='mse', optimizer=adam, metrics=['mae'])
    
    return model
    # $CHALLENGIFY_END

In [17]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

def fit_model(X_train, y_train, verbose=1):
    
    # Define hyperparameter space to search over
    param_grid = {
        'n_lstm': [32, 64, 128],
        'activation': ['tanh', 'relu'],
        'learning_rate': [0.001, 0.01, 0.1],
        'dropout_rate': [0.0, 0.2, 0.4],
        'batch_size': [32, 64, 128],
        'epochs': [50, 100, 150]
    }
    
    # Define the model
    def build_model(n_lstm, activation, learning_rate, dropout_rate):
        # Normalization
        normalizer = tf.keras.layers.Normalization()
        normalizer.adapt(X_train)
        
        # RNN architecture
        model = models.Sequential()
        model.add(normalizer)
        model.add(layers.LSTM(n_lstm, 
                              activation=activation,
                              dropout=dropout_rate,
                              recurrent_dropout=dropout_rate))
        model.add(layers.Dense(y_train.shape[1], activation='linear'))
        
        # Compiler
        optimizer = optimizers.Adam(learning_rate=learning_rate)
        model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
        
        return model
    
    # Randomized search to find best hyperparameters
    model = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model)
    search = RandomizedSearchCV(model, param_grid, cv=3, verbose=0, n_iter=10)
    search.fit(X_train, y_train)
    
    # Print best hyperparameters
    print("Best hyperparameters:", search.best_params_)
    
    # Build and fit model with best hyperparameters
    best_params = search.best_params_
    model = build_model(best_params['n_lstm'], best_params['activation'], 
                        best_params['learning_rate'], best_params['dropout_rate'])
    
    es = EarlyStopping(monitor = "val_loss",
                      patience = 5,
                      mode = "min",
                      restore_best_weights = True)

    history = model.fit(X_train, y_train,
                        validation_split = 0.3,
                        shuffle = False,
                        batch_size = best_params['batch_size'],
                        epochs = best_params['epochs'],
                        callbacks = [es],
                        verbose = verbose)

    return model, history.history

In [18]:
def plot_history(history):
    
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    # --- LOSS: MSE --- 
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('MSE')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].legend(['Train', 'Validation'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- METRICS:MAE ---
    
    ax[1].plot(history.history['mae'])
    ax[1].plot(history.history['val_mae'])
    ax[1].set_title('MAE')
    ax[1].set_ylabel('MAE')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Validation'], loc='best')
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
                        
    return ax

### Defining global variables

In [19]:
# Let's define the global variables of our dataset
#TARGET = target
N_TARGETS = 1
N_FEATURES = 41

Defining the folds informations

In [20]:
# --------------------------------------------------- #
# Let's consider FOLDS with a length of 3 years       #
# (2 years will be used for train, 1 for test!)       #
# --------------------------------------------------- #

FOLD_LENGTH = 24*365 * 3 # every hour
                        # three years

# --------------------------------------------------- #
# Let's consider FOLDS starting every trimester       #
# --------------------------------------------------- #
    
FOLD_STRIDE = 24*91 # every hour
                   # 1 quarter = 91 days

# --------------------------------------------------- #
# Let's consider a train-test-split ratio of 2/3      #
# --------------------------------------------------- #

TRAIN_TEST_RATIO = 0.66

And the input length

In [21]:
INPUT_LENGTH = 24 * 7 # records every hour
                      # one week

### Getting the dataframe

In [22]:
target = 'eolien'
#target = 'solaire'
#target = 'consommation'
TARGET = target

In [23]:
data = WeatherEnergy(limit=-1,
                     offset=0,
                     refine='Hauts-de-France', # for wind
                     #refine='Nouvelle-Aquitaine', # for sun
                     #refine='Hauts-de-France', # for consumption
                     target = target,
                     city=['Heudicourt','Bucy-les-Pierrepont','Riencourt'], # for wind
                     #city=['Cestas'], # for sun
                     #city=['Lille','Château-Thierry','Laon','Amiens'], # for consumption
                     years=10)

In [24]:
merged_df = data.merged()

In [25]:
processed_df = FeaturePreprocessing(merged_df,target)

In [26]:
df = processed_df.get_period_day()

In [27]:
df.head()

,standardscaler__temperature_2m,standardscaler__dewpoint_2m,standardscaler__apparent_temperature,standardscaler__pressure_msl,standardscaler__surface_pressure,standardscaler__Wx_10,standardscaler__Wx_100,standardscaler__Wy_10,standardscaler__Wy_100,standardscaler__windgusts_10m,standardscaler__soil_temperature_0_to_7cm,standardscaler__soil_temperature_7_to_28cm,standardscaler__soil_temperature_28_to_100cm,standardscaler__soil_temperature_100_to_255cm,standardscaler__soil_moisture_0_to_7cm,standardscaler__soil_moisture_7_to_28cm,standardscaler__soil_moisture_28_to_100cm,standardscaler__soil_moisture_100_to_255cm,robustscaler__cloudcover,robustscaler__cloudcover_low,robustscaler__cloudcover_mid,robustscaler__cloudcover_high,powertransformer__relativehumidity_2m,powertransformer__precipitation,powertransformer__rain,powertransformer__snowfall,powertransformer__shortwave_radiation,powertransformer__direct_radiation,powertransformer__direct_normal_irradiance,powertransformer__diffuse_radiation,powertransformer__et0_fao_evapotranspiration,powertransformer__vapor_pressure_deficit,season_Fall,season_Spring,season_Summer,season_Winter,weekday_Weekday,weekday_Weekend,period_Afternoon,period_Morning,period_Night,eolien
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-03-18 00:00:00,-1.3,-1.2,-1.3,-2.5,-2.5,-0.7,-0.7,-1.1,-1.3,0.7,-1.2,-1.3,-1.7,-1.8,1.3,1.4,1.1,1.5,-0.8,-0.3,-0.3,-0.3,0.6,-0.6,-0.6,-0.1,-1.0,-0.9,-0.9,-1.0,-0.8,-0.9,0,0,0,1,1,0,0,0,1,698.0
2013-03-18 01:00:00,-1.4,-1.3,-1.4,-2.4,-2.5,-0.8,-0.9,-0.7,-1.0,0.3,-1.3,-1.3,-1.7,-1.8,1.3,1.3,1.1,1.5,-0.8,-0.4,-0.3,-0.3,0.7,-0.6,-0.6,-0.1,-1.0,-0.9,-0.9,-1.0,-0.9,-1.1,0,0,0,1,1,0,0,0,1,586.0
2013-03-18 02:00:00,-1.5,-1.4,-1.4,-2.4,-2.5,-0.9,-1.0,-0.3,-0.7,-0.0,-1.4,-1.3,-1.7,-1.8,1.2,1.3,1.1,1.5,-0.5,-0.3,-0.0,-0.1,0.8,-0.6,-0.6,-0.1,-1.0,-0.9,-0.9,-1.0,-1.0,-1.1,0,0,0,1,1,0,0,0,1,454.0
2013-03-18 03:00:00,-1.6,-1.5,-1.5,-2.4,-2.5,-0.8,-1.1,-0.0,-0.4,-0.2,-1.5,-1.3,-1.7,-1.8,1.2,1.3,1.1,1.5,0.1,-0.1,0.5,0.4,0.9,-0.6,-0.6,-0.1,-1.0,-0.9,-0.9,-1.0,-1.0,-1.2,0,0,0,1,1,0,0,0,1,350.0
2013-03-18 04:00:00,-1.7,-1.6,-1.5,-2.4,-2.5,-0.9,-1.2,0.2,-0.0,-0.3,-1.5,-1.4,-1.7,-1.8,1.2,1.3,1.1,1.5,0.5,-0.2,1.2,0.9,0.9,-0.6,-0.6,-0.1,-1.0,-0.9,-0.9,-1.0,-1.0,-1.2,0,0,0,1,1,0,0,1,0,326.0


In [28]:
#df_scaled = df = pd.read_csv('raw_data/wind_energy_1.csv',index_col=0)

### Getting the folds

In [29]:
folds = get_folds(df, FOLD_LENGTH, FOLD_STRIDE)

print(f'The function generated {len(folds)} folds.')
print(f'Each fold has a shape equal to {folds[0].shape}.')

The function generated 29 folds.
Each fold has a shape equal to (26280, 42).


In [30]:
fold = folds[-1]
fold

,standardscaler__temperature_2m,standardscaler__dewpoint_2m,standardscaler__apparent_temperature,standardscaler__pressure_msl,standardscaler__surface_pressure,standardscaler__Wx_10,standardscaler__Wx_100,standardscaler__Wy_10,standardscaler__Wy_100,standardscaler__windgusts_10m,standardscaler__soil_temperature_0_to_7cm,standardscaler__soil_temperature_7_to_28cm,standardscaler__soil_temperature_28_to_100cm,standardscaler__soil_temperature_100_to_255cm,standardscaler__soil_moisture_0_to_7cm,standardscaler__soil_moisture_7_to_28cm,standardscaler__soil_moisture_28_to_100cm,standardscaler__soil_moisture_100_to_255cm,robustscaler__cloudcover,robustscaler__cloudcover_low,robustscaler__cloudcover_mid,robustscaler__cloudcover_high,powertransformer__relativehumidity_2m,powertransformer__precipitation,powertransformer__rain,powertransformer__snowfall,powertransformer__shortwave_radiation,powertransformer__direct_radiation,powertransformer__direct_normal_irradiance,powertransformer__diffuse_radiation,powertransformer__et0_fao_evapotranspiration,powertransformer__vapor_pressure_deficit,season_Fall,season_Spring,season_Summer,season_Winter,weekday_Weekday,weekday_Weekend,period_Afternoon,period_Morning,period_Night,eolien
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-03-09 00:00:00,-0.8,-0.6,-0.8,-0.6,-0.6,-0.5,-0.4,-1.1,-1.4,0.2,-0.9,-0.8,-1.0,-1.0,1.1,1.0,1.6,2.1,-0.2,-0.1,0.3,-0.2,0.8,-0.6,-0.6,-0.1,-1.0,-0.9,-0.9,-1.0,-0.9,-0.9,0,0,0,1,1,0,0,0,1,"1,933.0"
2020-03-09 01:00:00,-0.9,-0.6,-0.8,-0.6,-0.6,-0.5,-0.5,-1.1,-1.3,0.3,-0.9,-0.8,-1.0,-1.0,1.1,1.0,1.6,2.1,0.0,0.1,0.4,-0.2,0.8,-0.6,-0.6,-0.1,-1.0,-0.9,-0.9,-1.0,-0.9,-1.0,0,0,0,1,1,0,0,0,1,"1,973.0"
2020-03-09 02:00:00,-0.9,-0.6,-0.9,-0.5,-0.6,-0.6,-0.5,-1.2,-1.3,0.4,-0.9,-0.8,-1.0,-1.0,1.0,1.0,1.6,2.1,0.1,0.3,0.3,-0.2,0.9,-0.6,-0.6,-0.1,-1.0,-0.9,-0.9,-1.0,-0.9,-1.0,0,0,0,1,1,0,0,0,1,"2,005.0"
2020-03-09 03:00:00,-0.9,-0.7,-0.9,-0.5,-0.6,-0.5,-0.5,-1.3,-1.4,0.4,-0.9,-0.9,-1.0,-1.0,1.0,1.0,1.6,2.1,0.2,0.5,0.1,-0.3,0.9,-0.6,-0.6,-0.1,-1.0,-0.9,-0.9,-1.0,-0.9,-1.0,0,0,0,1,1,0,0,0,1,"1,922.0"
2020-03-09 04:00:00,-0.9,-0.6,-0.9,-0.5,-0.5,-0.4,-0.4,-1.4,-1.5,0.4,-0.9,-0.9,-1.0,-1.0,1.0,1.0,1.6,2.1,0.4,0.7,0.2,-0.3,0.8,-0.6,-0.6,-0.1,-1.0,-0.9,-0.9,-1.0,-0.9,-0.9,0,0,0,1,1,0,0,1,0,"1,984.0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-09 16:00:00,0.2,0.2,-0.0,-2.2,-2.2,-1.8,-1.7,-0.8,-0.7,1.8,-0.1,-0.7,-1.3,-1.2,1.1,1.1,0.7,0.9,0.6,-0.0,1.0,0.9,-0.5,2.2,2.2,-0.1,1.1,1.1,1.1,1.2,1.0,0.5,0,0,0,1,1,0,1,0,0,"3,570.0"
2023-03-09 17:00:00,-0.0,0.3,-0.1,-2.3,-2.3,-1.9,-1.8,-0.7,-0.7,1.3,-0.2,-0.6,-1.3,-1.2,1.2,1.1,0.7,0.9,0.6,-0.1,1.2,1.0,0.5,2.2,2.2,-0.1,0.7,0.1,0.8,0.9,0.3,-0.3,0,0,0,1,1,0,1,0,0,"3,532.0"
2023-03-09 18:00:00,-0.1,0.3,-0.1,-2.4,-2.4,-1.9,-1.9,-0.7,-0.7,1.4,-0.3,-0.6,-1.3,-1.2,1.3,1.1,0.7,0.9,0.7,0.4,1.5,1.0,0.7,2.2,2.2,-0.1,0.5,-0.4,0.0,0.6,-0.0,-0.5,0,0,0,1,1,0,1,0,0,"3,531.0"


### Spliting the train and test samples

In [31]:
(fold_train, fold_test) = train_test_split(fold, TRAIN_TEST_RATIO, INPUT_LENGTH)

In [32]:
# INPUT X
print(f'N_FEATURES = {N_FEATURES}')
print(f'INPUT_LENGTH = {INPUT_LENGTH} timesteps = {int(INPUT_LENGTH/24)} days = {int(INPUT_LENGTH/24/7)} weeks')

N_FEATURES = 41
INPUT_LENGTH = 168 timesteps = 7 days = 1 weeks


In [33]:
# TARGET Y
print(f'N_TARGETS = {N_TARGETS}')

# Let's only predict 1 value ahead of us
OUTPUT_LENGTH = 1
print(f'OUTPUT_LENGTH = {OUTPUT_LENGTH}')

N_TARGETS = 1
OUTPUT_LENGTH = 1


### Getting sequences

#### Random based

In [34]:
X_train_i, y_train_i = get_Xi_yi(fold_train, INPUT_LENGTH, OUTPUT_LENGTH)
X_test_i, y_test_i = get_Xi_yi(fold_test, INPUT_LENGTH, OUTPUT_LENGTH)

In [35]:
X_last, y_last = get_Xi_yi(fold_test, input_length=len(fold_test)-1, output_length=OUTPUT_LENGTH)
assert y_last.values == fold_test.iloc[-1,:][TARGET]

#### Not random 

In [36]:
N_TRAIN = 6666 # number_of_sequences_train
N_TEST =  3333 # number_of_sequences_test

X_train, y_train = get_X_y(fold_train, N_TRAIN, INPUT_LENGTH, OUTPUT_LENGTH)
X_test, y_test = get_X_y(fold_test, N_TEST, INPUT_LENGTH, OUTPUT_LENGTH)

In [37]:
SEQUENCE_STRIDE = 1

### Running the model

In [ ]:
# 1 - Initialising the RNN model
# ====================================

model = init_model(X_train, y_train)
model.summary()

# 2 - Training
# ====================================
model, history = fit_model(X_train,y_train)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, None, 41)         83        
 ion)                                                            
                                                                 
 lstm_1 (LSTM)               (None, 64)                27136     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 27,284
Trainable params: 27,201
Non-trainable params: 83
_________________________________________________________________


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Epoch 1/50
70/70 [==============================] - 12s 146ms/step - loss: 2594327.0000 - mae: 1131.3875
Epoch 2/50
70/70 [==============================] - 11s 157ms/step - loss: 2396751.0000 - mae: 1057.8427
Epoch 3/50
70/70 [==============================] - 10s 148ms/step - loss: 2226110.7500 - mae: 1003.7732
Epoch 4/50
70/70 [==============================] - 10s 148ms/step - loss: 2078541.3750 - mae: 962.2779
Epoch 5/50
70/70 [==============================] - 10s 150ms/step - loss: 1950644.3750 - mae: 931.7627
Epoch 6/50
70/70 [==============================] - 11s 153ms/step - loss: 1840414.6250 - mae: 909.1036
Epoch 7/50
70/70 [==============================] - 11s 158ms/step - loss: 1745816.3750 - mae: 893.2797
Epoch 8/50
70/70 [==============================] - 11s 151ms/step - loss: 1665553.1250 - mae: 882.0514
Epoch 9/50
70/70 [==============================] - 11s 152ms/step - loss: 1597514.3750 - mae: 875.3997
Epoch 10/50
70/70 [==============================] - 11s 150m

In [ ]:
plot_history(history);

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
import math

In [ ]:
print('R2 Score: ', r2_score(y_test.reshape((3333,1)), y_pred))
print('MSE Score: ', mean_squared_error(y_test.reshape((3333,1)), y_pred))
print('MAE Score: ', mean_absolute_error(y_test.reshape((3333,1)), y_pred))
print('MAPS error: ', mean_absolute_percentage_error(y_test.reshape((3333,1)), y_pred))
print('RMSE Score: ', math.sqrt(mean_squared_error(y_test.reshape((3333,1)), y_pred)))

In [ ]:
# eolien: r2 = 0.84
# solaire: r2 = 0.90

In [ ]:
model.save('saved_model/cons_model')

INFO:tensorflow:Assets written to: saved_model/cons_model/assets


INFO:tensorflow:Assets written to: saved_model/cons_model/assets


### Cross-validation

In [ ]:
res = model.evaluate(X_test, y_test)
print(f"The LSTM MAE on the test set is equal to {round(res[1],2)}")

In [ ]:
from tensorflow.keras.layers import Lambda

def init_baseline():

    # $CHALLENGIFY_BEGIN
    model = models.Sequential()
    model.add(layers.Lambda(lambda x: x[:,-1,1,None]))

    adam = optimizers.Adam(learning_rate=0.02)
    model.compile(loss='mse', optimizer=adam, metrics=["mae"])

    return model
    # $CHALLENGIFY_END

In [ ]:
baseline_model = init_baseline()
baseline_score = baseline_model.evaluate(X_test, y_test)
print(f"- The Baseline MAE on the test set is equal to {round(baseline_score[1],2)}")

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

def cross_validate_baseline_and_lstm():
    '''
    This function cross-validates 
    - the "last seen value" baseline model
    - the RNN model
    '''
    
    list_of_mae_baseline_model = []
    list_of_mae_recurrent_model = []
    
    # 0 - Creating folds
    # =========================================    
    folds = get_folds(df, FOLD_LENGTH, FOLD_STRIDE)
    
    for fold_id, fold in enumerate(folds):
        
        # 1 - Train/Test split the current fold
        # =========================================
        (fold_train, fold_test) = train_test_split(fold, TRAIN_TEST_RATIO, INPUT_LENGTH)                   

        X_train, y_train = get_X_y(fold_train, N_TRAIN, INPUT_LENGTH, OUTPUT_LENGTH)
        X_test, y_test = get_X_y(fold_test, N_TEST, INPUT_LENGTH, OUTPUT_LENGTH)
        
        # 2 - Modelling
        # =========================================
        
        ##### Baseline Model
        baseline_model = init_baseline()
        mae_baseline = baseline_model.evaluate(X_test, y_test, verbose=0)[1]
        list_of_mae_baseline_model.append(mae_baseline)
        print("-"*50)
        print(f"MAE baseline fold n°{fold_id} = {round(mae_baseline, 2)}")

        ##### LSTM Model
        model = init_model(X_train, y_train)
        es = EarlyStopping(monitor = "val_mae",
                           mode = "min",
                           patience = 2, 
                           restore_best_weights = True)
        history = model.fit(X_train, y_train,
                            validation_split = 0.3,
                            shuffle = False,
                            batch_size = 32,
                            epochs = 50,
                            callbacks = [es],
                            verbose = 0)
        res = model.evaluate(X_test, y_test, verbose=0)
        mae_lstm = res[1]
        list_of_mae_recurrent_model.append(mae_lstm)
        print(f"MAE LSTM fold n°{fold_id} = {round(mae_lstm, 2)}")
        
        ##### Comparison LSTM vs Baseline for the current fold
        print(f"🏋🏽‍♂️ improvement over baseline: {round((1 - (mae_lstm/mae_baseline))*100,2)} % \n")

    return list_of_mae_baseline_model, list_of_mae_recurrent_model

In [ ]:
%%time
# WARNING : it takes 75 minutes to run this cell 
mae_baselines, mae_lstms = cross_validate_baseline_and_lstm()

## Forecast data

In [ ]:
from prophecy.get_data_forecast import WeatherForecast
from prophecy.process_forecast_data import ForecastDataframe
from prophecy.get_data import WeatherEnergy
import tensorflow as tf

### Getting the dataframes

In [ ]:
wind_df = ForecastDataframe('Heudicourt').return_scaled_forecast()

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [ ]:
sun_df = ForecastDataframe('Cestas').return_scaled_forecast()

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [ ]:
city = 'Amiens'

In [ ]:
cons_df = ForecastDataframe(city).return_scaled_forecast()

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


Still columns missing

### Loading the models

In [ ]:
wind_model = tf.keras.models.load_model('saved_model/wind_model')

In [ ]:
sun_model = tf.keras.models.load_model('saved_model/sun_model')

In [ ]:
cons_model = tf.keras.models.load_model('saved_model/cons_model')

### Making the predictions

In [ ]:
wind_predict = wind_model.predict(wind_df)

ValueError: in user code:

    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/opt/conda/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer 'normalization_17' (type Normalization).
    
    Dimensions must be equal, but are 41 and 42 for '{{node sequential_17/normalizat

In [ ]:
sun_predict = sun_model.predict(sun_df_prep)

In [ ]:
cons_predict = cons_model.predict(cons_df_prep)

### Computing the recommendations

In [ ]:
eolien_data = WeatherEnergy(limit=-1, offset=0, city='Amiens', target='eolien', refine='Hauts-de-France')

In [ ]:
eolien_df = eolien_data.get_energy_production().asfreq('h').reset_index().drop(columns='time')

In [ ]:
eolien_df["eolien"]

0           NaN
1       1,062.0
2         951.0
3         769.0
4         637.0
          ...  
89467     861.0
89468   1,019.0
89469       NaN
89470       NaN
89471       NaN
Name: eolien, Length: 89472, dtype: float64

In [ ]:
transformed_ts1 = pd.Series(np.log(pd.concat([eolien_df["eolien"], wind_predict["eolien"][-1:]]))).diff().dropna()

In [ ]:
age_col_index = df.columns.get_loc('season_Fall')
age_col_index

32